In [48]:
import base64
import mimetypes # For guessing MIME type
from openai import OpenAI
from pathlib import Path

# Set the API key and base URL to your vLLM server
openai_api_key = "EMPTY"  # vLLM typically doesn't require an API key by default
openai_api_base = "http://localhost:7555/v1" # Default vLLM server address

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

with open("prompts/general_ocr_prompt.txt") as f:
    ocr_prompt= f.read()

In [ ]:
# vllm serve /home/ubuntu/models/Qwen/Qwen2.5-VL-32B-Instruct-AWQ --port 7555 --quantization awq --dtype float16 --allowed-local-media-path /home/ubuntu/ocr/vllm/benchmark/jpg

In [49]:
!curl http://localhost:7555/v1/models

{"object":"list","data":[{"id":"/home/ubuntu/models/Qwen/Qwen2.5-VL-32B-Instruct-AWQ","object":"model","created":1747912126,"owned_by":"vllm","root":"/home/ubuntu/models/Qwen/Qwen2.5-VL-32B-Instruct-AWQ","parent":null,"max_model_len":128000,"permission":[{"id":"modelperm-3e0c57d7a9294c19b9f15a80638c6137","object":"model_permission","created":1747912126,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [50]:
ocr_prompt

'Act as an OCR assistant. Analyze the provided <|image|> image and:\n1. Identify and transcribe all visible text in the image exactly as it appears.\n2. Preserve the original line breaks, spacing, and formatting from the image.\n3. Output only the transcribed text, line by line, without adding any commentary or explanations or special characters.'

In [51]:
def get_ocr(file_path):
    # Example for chat completions API (if supported by the model)
    try:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {"url": "file://" + str(file_path)}},
                    {"type": "text", "text": ocr_prompt}
                ]
            }
        ]
        chat_response = client.chat.completions.create(
            model="/home/ubuntu/models/Qwen/Qwen2.5-VL-32B-Instruct-AWQ", # Specify the model loaded on your vLLM server
            messages=messages
        )
        print("Chat completion result:", chat_response)
    except Exception as e:
        print(f"An error occurred with chat completions: {e}")
        print("Note: Some models may not support the 'system' role or the chat API.") 



In [ ]:
get_ocr("/home/ubuntu/ocr/vllm/benchmark/jpg/1_page_1.jpg")

An error occurred with chat completions: Error code: 400 - {'object': 'error', 'message': 'The file path /home/den/Documents/Nornikel/vllm/benchmark/original/document_000/page_1.jpg must be a subpath of `--allowed-local-media-path` /home/ubuntu/ocr/vllm/benchmark/jpg.', 'type': 'BadRequestError', 'param': None, 'code': 400}
Note: Some models may not support the 'system' role or the chat API.


In [54]:
# destination_folder = Path("/home/ubuntu/ocr/vllm/benchmark/renamed")
# destination_folder.mkdir(parents=True, exist_ok=True)

# for i, file_path in enumerate(Path("/home/ubuntu/ocr/vllm/benchmark/original").glob("*")):
#     new_name = f"document_{i:03d}.pdf"
#     new_path = destination_folder / new_name

#     # Move the file to the destination folder with the new name
#     file_path.rename(new_path)
#     print(f"Moved: {file_path.name} → {new_name}")
#     print(i)
